In [31]:
import xarray as xr
from utils.degree_day_equations import *
from utils.net_cdf_functions import *
from utils.processing_functions import *




ModuleNotFoundError: No module named 'utils'

I have some .nc data on a server I'd like to access with pydap and xarray. I want only the tmin and tmax variables for some number of dates - that I want to specify with a start date and a n_days. IF n_days is None, I want to select all data to the present. 

Sometimes I want data within some bounding box and sometimes I want data for only a certain point. 

here is an example dataset for jan 1st 2016 - https://thredds.climate.ncsu.edu/thredds/catalog/prism/daily/combo/2016/catalog.html?dataset=prism/daily/combo/2016/PRISM_combo_20160101.nc

the long string at the end is the year, month and date 

can you write me a function to call the data I want and collect into xarray?

<xarray.Dataset> Size: 28MB
Dimensions:    (t: 1, longitude: 1405, latitude: 621)
Coordinates:
  * t          (t) datetime64[ns] 8B 2016-01-01
  * longitude  (longitude) float64 11kB -125.0 -125.0 -124.9 ... -66.54 -66.5
  * latitude   (latitude) float64 5kB 49.92 49.88 49.83 ... 24.17 24.12 24.08
Data variables:
    ppt        (t, latitude, longitude) float64 7MB ...
    tmean      (t, latitude, longitude) float64 7MB ...
    tmin       (t, latitude, longitude) float64 7MB ...
    tmax       (t, latitude, longitude) float64 7MB ...
Attributes:
    DODS_EXTRA.Unlimited_Dimension:  t
<xarray.Dataset> Size: 28MB
Dimensions:    (t: 1, longitude: 1405, latitude: 621)
Coordinates:
  * t          (t) datetime64[ns] 8B 2016-01-02
  * longitude  (longitude) float64 11kB -125.0 -125.0 -124.9 ... -66.54 -66.5
  * latitude   (latitude) float64 5kB 49.92 49.88 49.83 ... 24.17 24.12 24.08
Data variables:
    ppt        (t, latitude, longitude) float64 7MB ...
    tmean      (t, latitude, longitude)

In [16]:
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta

def fetch_thredds_data(base_url, start_date, n_days=None, bbox=None, point=None):
    """
    Fetch data from a THREDDS server and collect it into an xarray Dataset.
    
    Parameters:
    - base_url: str, base URL of the THREDDS server
    - start_date: str, start date in the format 'YYYY-MM-DD'
    - n_days: int or None, number of days to fetch data for. If None, fetch all data to the present.
    - bbox: tuple or None, bounding box in the format (lon_min, lon_max, lat_min, lat_max)
    - point: tuple or None, specific point in the format (lon, lat)
    
    Returns:
    - xarray.Dataset containing the requested data
    """
    # Convert start_date to datetime
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    
    # Calculate end_date
    if n_days is None:
        end_date = datetime.now()
    else:
        end_date = start_date + timedelta(days=n_days)
    
    # Generate list of dates
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Initialize an empty list to store datasets
    datasets = []
    
    # Loop through each date and fetch the data
    for date in dates:
        date_str = date.strftime('%Y%m%d')
        url = f"{base_url}/PRISM_combo_{date_str}.nc"
        
        # Open the dataset using xarray
        ds = xr.open_dataset(url)
        #print(ds)
        # Select tmin and tmax variables
        ds = ds[['tmin', 'tmax']]
        print(ds)
        # Apply bounding box or point selection if specified
        if bbox is not None:
            lon_min, lon_max, lat_min, lat_max = bbox
            ds = ds.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_min, lat_max))
        elif point is not None:
            lon, lat = point
            ds = ds.sel(longitude=lon, latitude=lat, method='nearest')
        #print(ds)
        # Append the dataset to the list
        datasets.append(ds)
    
    # Combine all datasets into a single xarray Dataset
    combined_ds = xr.concat(datasets, dim='time')
    
    return combined_ds

# Example usage
base_url = "https://thredds.climate.ncsu.edu/thredds/dodsC/prism/daily/combo/2016"
start_date = "2016-01-01"
n_days = 10
bbox = (-125, -124, 48, 47)  # Example bounding box (lon_min, lon_max, lat_min, lat_max)
point = None  # Example point (lon, lat)

data = fetch_thredds_data(base_url, start_date, n_days, bbox, point)
print(data)

<xarray.Dataset> Size: 14MB
Dimensions:    (t: 1, latitude: 621, longitude: 1405)
Coordinates:
  * t          (t) datetime64[ns] 8B 2016-01-01
  * longitude  (longitude) float64 11kB -125.0 -125.0 -124.9 ... -66.54 -66.5
  * latitude   (latitude) float64 5kB 49.92 49.88 49.83 ... 24.17 24.12 24.08
Data variables:
    tmin       (t, latitude, longitude) float64 7MB ...
    tmax       (t, latitude, longitude) float64 7MB ...
Attributes:
    DODS_EXTRA.Unlimited_Dimension:  t
<xarray.Dataset> Size: 14MB
Dimensions:    (t: 1, latitude: 621, longitude: 1405)
Coordinates:
  * t          (t) datetime64[ns] 8B 2016-01-02
  * longitude  (longitude) float64 11kB -125.0 -125.0 -124.9 ... -66.54 -66.5
  * latitude   (latitude) float64 5kB 49.92 49.88 49.83 ... 24.17 24.12 24.08
Data variables:
    tmin       (t, latitude, longitude) float64 7MB ...
    tmax       (t, latitude, longitude) float64 7MB ...
Attributes:
    DODS_EXTRA.Unlimited_Dimension:  t
<xarray.Dataset> Size: 14MB
Dimensions:    (

In [ ]:
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta

def fetch_thredds_PRISM(base_url, start_date, n_days=None, bbox=None, point=None):
    """
    Fetch data from a THREDDS server and collect it into an xarray Dataset.
    
    Parameters:
    - base_url: str, base URL of the THREDDS server
    - start_date: str, start date in the format 'YYYY-MM-DD'
    - n_days: int or None, number of days to fetch data for. If None, fetch all data to the present.
    - bbox: tuple or None, bounding box in the format (lon_min, lon_max, lat_min, lat_max)
    - point: tuple or None, specific point in the format (lon, lat)
    
    Returns:
    - xarray.Dataset containing the requested data
    """
    # Convert start_date to datetime
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    
    # Calculate end_date
    if n_days is None:
        end_date = datetime.now()
    else:
        end_date = start_date + timedelta(days=n_days)
    
    # Generate list of dates
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Initialize an empty list to store datasets
    datasets = []
    
    # Loop through each date and fetch the data
    for date in dates:
        date_str = date.strftime('%Y%m%d')
        url = f"{base_url}/PRISM_combo_{date_str}.nc"
        
        # Open the dataset using xarray
        ds = xr.open_dataset(url)
        
        # Select tmin and tmax variables
        ds = ds[['tmin', 'tmax']]
        
        # Debug: Print the dataset coordinates before subsetting
        #print(ds.coords)
        
        # Apply bounding box or point selection if specified
        if bbox is not None:
            lon_min, lon_max, lat_min, lat_max = bbox
            # Ensure latitude slice is in the correct order
            ds = ds.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_max, lat_min))
        elif point is not None:
            lon, lat = point
            ds = ds.sel(longitude=lon, latitude=lat, method='nearest')
        
        # Debug: Print the dataset coordinates after subsetting
        #print(ds.coords)
        
        # Append the dataset to the list
        datasets.append(ds)
    
    # Combine all datasets into a single xarray Dataset
    combined_ds = xr.concat(datasets, dim='t')
    
    return combined_ds

# Example usage
base_url = "https://thredds.climate.ncsu.edu/thredds/dodsC/prism/daily/combo/2016"
start_date = "2016-01-01"
n_days = 10
bbox = (-110, -109, 33, 34)  # Example bounding box (lon_min, lon_max, lat_min, lat_max)
point = None  # Example point (lon, lat)

data = fetch_thredds_data(base_url, start_date, n_days, bbox, point)
print(data)

KeyboardInterrupt: 

In [27]:
data = fetch_thredds_data(base_url, start_date, n_days, bbox = None, point = (-110,33))
data

<xarray.Dataset> Size: 280B
Dimensions:    (t: 11)
Coordinates:
  * t          (t) datetime64[ns] 88B 2016-01-01 2016-01-02 ... 2016-01-11
    longitude  float64 8B -110.0
    latitude   float64 8B 33.0
Data variables:
    tmin       (t) float64 88B -5.802 3.136 0.185 0.151 ... -3.037 -3.681 -3.155
    tmax       (t) float64 88B 13.74 11.96 12.63 13.79 ... 3.862 9.803 8.171
Attributes:
    DODS_EXTRA.Unlimited_Dimension:  t